# Model Training
Welcome to the 'Model Training and Prediction' notebook, a crucial facet of our project's data science pipeline. In this notebook, we offer a meticulous examination of our rigorous model development process. The pipeline starts by accepting training data, followed by fitting three distinct types of models to it: Random Forest, Gradient Boosted Tree, and XGBoost. The initial stages include encoding categorical variables and executing Recursive Feature Elimination (RFE) for feature selection. This is succeeded by the application of genetic algorithms to hyperparameter tuning, operating in tandem with a cross-validation routine. Subsequently, the best model is selected based on the highest accuracy score. Finally, the selected model is saved for later use in predicting the outcomes for NRL matches.

## Set up Environment
he initial section of the code is dedicated to preparing the environment for our model training pipeline. Key Python libraries, including sys, pandas, sqlite3, pathlib, and numpy, are imported to manage system parameters, perform data operations, handle database connectivity, manage file paths, and conduct numerical operations respectively. Additionally, the path of our custom modules, residing in the 'model-training/functions' directory, is appended to the system path, enabling us to import the modelling_functions and training_config modules. These modules contain custom functions and configuration settings essential for the subsequent phases of data preprocessing, model training, and prediction. This setup ensures all necessary tools and functions are readily available for the pipeline's operations.

In [5]:
# %pip install pandas numpy
#!pip3 install -r requirements.txt
#%pip install -r /Users/levonrush/Documents/GitHub/footy-tipper/requirements.txt



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
import pandas as pd
import sqlite3
import pathlib
import numpy as np
from joblib import dump

sys.path.append("functions") 
import modelling_functions as mf
import model_properties as mp
import training_config as tc

## Get data
Our process starts by establishing the root directory of the project and constructing the relative path to the 'footy-tipper-db.sqlite' database located within the 'data' directory. We then connect to this SQLite database and use a SQL query housed in the 'footy_tipping_data.sql' file, found in the 'sql' directory, to extract the required data. This data is loaded into a pandas DataFrame, footy_tipping_data, serving as the basis for our subsequent modeling activities. Upon successful extraction of the data, we ensure the database connection is closed, maintaining good coding practice and resource management.

In [7]:
# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"

# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file
with open('sql/training_data.sql', 'r') as file:
    query = file.read()

footy_tipping_data = pd.read_sql_query(query, con)

# Don't forget to close the connection
con.close()

footy_tipping_data

,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,matchup_form,state_of_origin,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.012111e+10,1.0,Round 1,1.0,Final,1.330600e+09,1.330560e+09,McDonald Jones Stadium,Newcastle,29189.0,...,0.0,0.0,1500.000000,1500.000000,0.000000,0.490595,0.483431,0.025974,0.007163,NaN
1,2.012111e+10,1.0,Round 1,2.0,Final,1.330686e+09,1.330646e+09,Bankwest Stadium,Sydney,11399.0,...,0.0,0.0,1500.000000,1500.000000,0.000000,0.490595,0.483431,0.025974,0.007163,NaN
2,2.012111e+10,1.0,Round 1,3.0,Final,1.330772e+09,1.330733e+09,Canberra Stadium,Canberra,7862.0,...,0.0,0.0,1500.000000,1500.000000,0.000000,0.490595,0.483431,0.025974,0.007163,NaN
3,2.012111e+10,1.0,Round 1,4.0,Final,1.330772e+09,1.330733e+09,Panthers Stadium,Penrith,9585.0,...,0.0,0.0,1500.000000,1500.000000,0.000000,0.490595,0.483431,0.025974,0.007163,NaN
4,2.012111e+10,1.0,Round 1,5.0,Final,1.330769e+09,1.330733e+09,1300SMILES Stadium,Townsville,16311.0,...,0.0,0.0,1500.000000,1500.000000,0.000000,0.490595,0.483431,0.025974,0.007163,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,2.023111e+10,27.0,Round 27,4.0,Pre Game,1.693667e+09,1.693631e+09,Suncorp Stadium,Brisbane,NaN,...,0.0,0.0,1485.770190,1510.073119,-24.302929,0.453002,0.513416,0.033582,-0.060414,NaN
2379,2.023111e+10,27.0,Round 27,5.0,Pre Game,1.693676e+09,1.693640e+09,BlueBet Stadium,Penrith,NaN,...,3.0,0.0,1522.360288,1522.046948,0.313340,0.491034,0.482992,0.025974,0.008042,NaN
2380,2.023111e+10,27.0,Round 27,6.0,Pre Game,1.693683e+09,1.693647e+09,Netstrata Jubilee Stadium,Sydney,NaN,...,3.0,0.0,1481.961590,1501.964281,-20.002691,0.458964,0.507454,0.033582,-0.048491,NaN
2381,2.023111e+10,27.0,Round 27,7.0,Pre Game,1.693750e+09,1.693714e+09,Cbus Super Stadium,Gold Coast,NaN,...,1.0,0.0,1503.571364,1468.534996,35.036368,0.533613,0.429781,0.036606,0.103832,NaN


## Modelling
During the modeling phase, we invoke the train_and_select_best_model function from our modelling_functions module. This function, by accepting our footy tipping data, predictor variables, outcome variable, and several configuration settings (including Recursive Feature Elimination (RFE) usage, cross-validation folds number, and optimization metric) from the training_config module, initiates the training of three distinct models: Random Forest, Gradient Boosted Tree, and XGBoost. Each model undergoes hyperparameter tuning via genetic algorithms and is evaluated through cross-validation. The best_model, defined by the superior performance on the chosen optimization metric, is selected and, together with X_inference (transformed feature matrix), label_encoder (for encoding categorical variables), and game_id_inference (for associating predictions with specific games), is returned, ready for the prediction phase.

In [6]:
best_model, X_inference, label_encoder, game_id_inference = mf.train_and_select_best_model(
    footy_tipping_data, tc.predictors, tc.outcome_var,
    tc.use_rfe, tc.num_folds, tc.opt_metric
)

best_model

Training model: XGBClassifier


Number of selected features: 77 out of 268
{'n_estimators': 59, 'learning_rate': 0.09657195668902026, 'max_depth': 3, 'subsample': 0.8070465353388098, 'colsample_bytree': 0.6473933830875238, 'gamma': 0.1946287499835999}
0.7270602337428829
Training model: RandomForestClassifier
Number of selected features: 34 out of 268
{'n_estimators': 195, 'max_features': 'sqrt', 'max_depth': 28, 'min_samples_split': 10, 'min_samples_leaf': 3, 'bootstrap': True}
0.7341424433123565
Training model: GradientBoostingClassifier
Number of selected features: 3 out of 268
{'n_estimators': 59, 'learning_rate': 0.08460174393084735, 'max_depth': 3, 'min_samples_split': 8, 'min_samples_leaf': 1, 'subsample': 0.9750793559032782, 'max_features': 'sqrt'}
0.7426530816102288


GASearchCV(crossover_probability=0.8,
           cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
           estimator=GradientBoostingClassifier(learning_rate=0.08460174393084735,
                                                max_features='sqrt',
                                                min_samples_split=8,
                                                n_estimators=59,
                                                subsample=0.9750793559032782),
           generations=100, keep_top_k=4, mutation_probability=0.1, n_jobs=-1,
           param_grid={'learning_rate': <s...
                       'min_samples_leaf': <sklearn_genetic.space.space.Integer object at 0x1372a4150>,
                       'min_samples_split': <sklearn_genetic.space.space.Integer object at 0x13729af90>,
                       'n_estimators': <sklearn_genetic.space.space.Integer object at 0x1372d1290>,
                       'subsample': <sklearn_genetic.space.space.Continuous object at 0x1372a4110>},
           population_size=10, return_train_score=True, scoring='accuracy',
           verbose=False)

In [ ]:
feature_importance_df = mp.get_sorted_feature_importance(best_model, X_inference.columns)
feature_importance_df

,Feature,Importance
0,team_head_to_head_odds_away,0.088276
1,team_head_to_head_odds_home,0.072208
2,home_elo_prob,0.041809
3,home_ground_advantage,0.029224
4,season_form_away,0.028061
...,...,...
72,start_hour,0.000000
73,position_diff,0.000000
74,team_home_Manly-Warringah Sea Eagles,0.000000
75,draw_rate_away,0.000000


## Save Model
Finally we save the trained machine learning model and label encoder into files for later use in prediciton.

In [ ]:
# Define the directory and filename where you want to save your model
model_dir = project_root / "models"

# Save the model and label encoder
dump(best_model, model_dir / "best_model.joblib")
dump(label_encoder, model_dir / "label_encoder.joblib")

NameError: name 'best_model' is not defined